In [ ]:
import time
import matplotlib.pyplot as plt
from io import BytesIO

from models.anomaly_detect import anomaly_detect  
from models.mysqldb import Mysqldb
from models.metric import Metric
from config import config

task_settings = {
    "data":{
        "metric_id": 26, 
        "granularity":"d1",
        "accum_items": [1, 4, 8, 12],
        # "dt_from": "2024-10-16",
        "dt_from": "",
        "last_items": 0
    },
    "anoms":{
        "direction": "both", 
        "max_anoms": 0.2, 
        "alpha": 0.01, 
        "piecewise_median_period_weeks": 2
    },
    "message_lvl":"all"
}

start_time = time.time()    
db = Mysqldb(config['db'])

for accum_items in task_settings['data']['accum_items']:

    mt = Metric.get_metric(db=db, 
                        id=task_settings['data']['metric_id'], 
                        granularity=task_settings['data']['granularity'])
    print("metric_name: ", mt.info['metric_name'] + ";  dp: ", mt.info['metric_dp'])

    dt_from = SysBf.tzdt_fr_str(task_settings['data']['dt_from'], "Europe/Moscow")
    # reliance = mt.get_data(accum_items=8, dt_from='', last_items=0) # 2024-06-01
    reliance = mt.get_data(accum_items=accum_items, 
                                    dt_from=tdt_from, 
                                    last_items=task_settings['data']['last_items'], tz_str="Europe/Moscow")
    res = reliance.squeeze()

    # direction options: pos | neg | both
    print('res items: ', len(res))
    if len(res)>0:
        anom_dict = anomaly_detect(res, max_anoms=0.2, direction='both', alpha=0.01, piecewise_median_period_weeks=2) 
        anoms_pos = anom_dict['anoms_pos']
        anoms_neg = anom_dict['anoms_neg']

        fig, ax = plt.subplots(1, figsize=(15,5))
        res.plot(ax=ax)
        if len(anoms_pos)>0:
            ax.scatter(anoms_pos.index.values, anoms_pos, color='green')
        if len(anoms_neg)>0:
            ax.scatter(anoms_neg.index.values, anoms_neg, color='red')
        ax.set_xlabel('Date time')
        ax.set_ylabel('Count')
        ax.set_title(mt.info['metric_name'] + ' [accum_items=' + str(accum_items) + ']')

        plot_file = BytesIO()
        fig.savefig(plot_file, format='png')
        plot_file.seek(0)

    else:
        print('No Data for metric!')    


print(round(time.time() - start_time, 4), 'sec')
